# Chapter 24. MapReduce

In [61]:
from __future__ import division
import math, random, re, datetime
from collections import defaultdict, Counter
from functools import partial
from naive_bayes import tokenize

[MapReduce](https://en.wikipedia.org/wiki/MapReduce) is a programming model for performing parallel processing on large data sets.  
Imagine that we have a (very large) collection that we would like to process in some way.  
For example, the items might be website logs, the texts of various books, image files, or anything else.  
A basic version of the MapReduce algorithm consists of the following steps:
1. Use a `mapper` function to turn each item into zero or more key-value pairs. This is often called the `map()` function, but Python already has a function called `map()`, and we don't want to confuse the two.
2. Collect together all of the pairs with identical keys.
3. Use a `reducer()` function on each collection of grouped values to produce output values for the corresponding key.  

Let's examine this process more concretely using an example that involves counting words.

## Example: Word Count

DataSciencester has grown to millions to users!  
This is great for your job security, but it makes routine analyses slightly more difficult.  
For example, your VP of Content wants to know what sorts of things people are talking about in their status updates.  
As a first attempt, you decide to count the words that appear, so that you can prepare a report on the most frequent ones.  
When you only had a few hundred users this was simple to do:

In [62]:
def word_count_old(documents):
    """ word count not using MapReduce """
    return Counter(word
                   for document in documents
                   for word in tokenize(document))

In [63]:
documents = ["data science", "big data", "science fiction"]
word_count_old(documents)

Counter({'big': 1, 'data': 2, 'fiction': 1, 'science': 2})

With millions of users the set of `documents` (status updates) is now too big to fit on your computer.  
If you can just fit this into the MapReduce model, you can use some "big data" infrastructure that your engineers have implemented.  
First, we need a function that turns a document into a sequence of key-value pairs.  
We'll want our output to be grouped by word, which means that the keys should be words.  
For each word, we'll use the value 1 to indicate that this pair corresponds to one occurrence of the word:

In [64]:
def wc_mapper(document):
    """ for each word in the document, return (word,1) """
    for word in tokenize(document):
        yield (word, 1)

wc_mapper_results = [result
                     for document in documents
                     for result in wc_mapper(document)]

wc_mapper_results

[('science', 1),
 ('data', 1),
 ('big', 1),
 ('data', 1),
 ('science', 1),
 ('fiction', 1)]

Skipping the "plumbing" step 2 (collect together all the pairs with identical keys) for the moment, imagine that for some word we've collected a list of the corresponding counts we have emitted.  
Then to produce the overall count for that word we can use:

In [65]:
def wc_reducer(word, counts):
    """ sum up the counts for a word """
    yield (word, sum(counts))

Now returning to step 2, we need to collect the results from `wc_mapper()` and feed them to `wc_reducer()`.  
Let's think about how we can do this on just one computer:

In [66]:
def word_count(documents):
    """ count the words in the input documents using MapReduce """
    # create a place to store the grouped values
    collector = defaultdict(list)
    
    for document in documents:
        for word, count in wc_mapper(document):
            collector[word].append(count)
       
    return [output
            for word, counts in collector.iteritems()
            for output in wc_reducer(word, counts)]

Now let's take our three documents:

In [67]:
documents = ["data science", "big data", "science fiction"]

Then `wc_mapper` applied to the first document yields the two pairs `("data", 1)` and `("science", 1)`.  
After we've gone through all three documents, the `collector` contains:

Then `wc_reducer()` produces the count for each word:

In [68]:
word_count(documents)

[('science', 2), ('fiction', 1), ('data', 2), ('big', 1)]

## Why MapReduce?

As mentioned earlier, the primary benefit of MapReduce is that it allows us to distribute computations by moving the processing to the data.  
Imagine that we want to word-count across billions of documents.  
Our original (non-MapReduce) approach requires the machine doing the processing to have access to every document.  
This means that the documents all need to either live on that macine or else be transferred to it during processing.  
More important, it means that the machine can only process one document at a time.  
Granted, you can write code to use multiple processors, multiple cores on the processor, and so on, but the documents still have to get to that machine.  
Now, imagine that our billions of documents are scattered across 100 machines.  
With the right infrastructure (and glossing over some of the details), we can do the following:  
- Have the machine run the mapper on its documents, producing lots of (key, value) pairs.
- Distribute those (key, value) pairs to a number of "reducing" machines, making sure that the pairs corresponding to any given key all end up on the same machine.
- Have each reducing machine group the pairs by key and then run the reduce on each set of values.
- Return each (key, output) pair. 

What is amazing about this is that it scales horizontally.  
If we double the number of machines, then (ignoring certain fixed costs of running a MapReduce system) our calculations will be executed approximately twice as fast.  
Each mapper machine will only need to do half as much work, and (assuming that there are enough distinct keys to further distribute the reducer work) the same is true for the reducer machines.

## MapReduce More Generally